In [1]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils

In [2]:
include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")

SystemError: SystemError: opening file "c:\\NTNU\\_Kurse TA\\Optimierung 2\\Julia_functions\\solution_animation.jl": No such file or directory

In [4]:
domain = (0,1,0,1)
partition = (100,100)
model = CartesianDiscreteModel(domain,partition) #|> simplexify
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1) ###### conformity correct?
Trialspace = TransientTrialFESpace(Testspace)                                # maybe add a function for/if Dirichlet conditions

Uspace = FESpace(model, reffe, conformity=:H1)

degree = 2*order                                                    # degree of the method used for approximating integrals over Ω
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)      										# make the measure dΩ
Γ = BoundaryTriangulation(model)                                    # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)   

χ(x,a,b)= a≤x≤b ? 1.0 : 0.0

q_pos(x) = χ(x[1], 0.9, 0.85) * χ(x[2], 0, 0.20)

ρ=1.0 #1.225
c = 1.0# c=1020.0
k = 1.0#1.0# k=15.0
h = 0.5# h=0.7
Tout(x,t)= 0.0*x[1]
Q(x,t) = 0.0#x[1] + x[2] #0.0#10.0*q_pos(x)
Tini(x) = 20.0 #1.0 * x[1] + 10.0
t0=0.0
tF=1.0
TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))
Δt = 0.001

function draw(ysol)
	fig, _ , plt = CairoMakie.plot(Ω, ysol, colormap=:plasma)               # plot of last state (numerical solution)
	CairoMakie.wireframe!(Ω, color=:black, linewidth=1)                        # add triangulation
	CairoMakie.Colorbar(fig[1,2], plt)                                         # add color bar
	display(fig)															  # display the plot
end
#draw(TIni)


draw (generic function with 1 method)

In [6]:
ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)

Q(t)=x->Q(x,t)
Tout(t)=x->Tout(x,t)

a_SE_tconst(t, dtT, ϕ) = ∫(c*dtT*ϕ*ρ)dΩ
# a_SE_tnonconst(t, T, ϕ) = (1)*∫(k * ∇(T) ⋅ ∇(ϕ) - tr(h*T*ϕ))dΩ
a_SE_tnonconst(t, T, ϕ) = ∫(k * ∇(T)⊙∇(ϕ))dΩ - ∫(h*T*ϕ)dΓ


l_SE(t, ϕ) = ∫(Q(t) * ϕ)dΩ - ∫(Tout(t) * ϕ * h)dΓ

op_SE = TransientLinearFEOperator((a_SE_tconst, a_SE_tnonconst), l_SE, Trialspace, Testspace, constant_forms=(true, true))

T = solve(solver, op_SE, t0, tF, TIni)

sol = []
count = 0
for (tn, uhn) in T
    if count%5 == 0
        push!(sol,uhn)
    end
    count += 1
    
end


In [7]:
if !isdir("tmp2")
    mkdir("tmp2")
  end
  
  createpvd("results2") do pvd
    pvd[0] = createvtk(Ω, "tmp2/results_0" * ".vtu", cellfields=["u" => TIni])
    for (tn, uhn) in T
      pvd[tn] = createvtk(Ω, "tmp2/results_$tn" * ".vtu", cellfields=["u" => uhn])
    end
  end

1002-element Vector{String}:
 "results2.pvd"
 "tmp2/results_0.vtu"
 "tmp2/results_0.001.vtu"
 "tmp2/results_0.002.vtu"
 "tmp2/results_0.003.vtu"
 "tmp2/results_0.004.vtu"
 "tmp2/results_0.005.vtu"
 "tmp2/results_0.006.vtu"
 "tmp2/results_0.007.vtu"
 "tmp2/results_0.008.vtu"
 ⋮
 "tmp2/results_0.9920000000000008.vtu"
 "tmp2/results_0.9930000000000008.vtu"
 "tmp2/results_0.9940000000000008.vtu"
 "tmp2/results_0.9950000000000008.vtu"
 "tmp2/results_0.9960000000000008.vtu"
 "tmp2/results_0.9970000000000008.vtu"
 "tmp2/results_0.9980000000000008.vtu"
 "tmp2/results_0.9990000000000008.vtu"
 "tmp2/results_1.0000000000000007.vtu"

In [5]:
fig, ax, hm = plot(sol[1])
Colorbar(fig[1,2], hm)
display(fig)

UndefVarError: UndefVarError: `sol` not defined

In [149]:
#solution_animation(sol,30)